In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [1]:
import torch
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import cv2
import config
from importlib import reload

In [202]:


def gen_anchors(feature_size, scale):
    heights = [11, 16, 23, 33, 48, 68, 97, 139, 198, 283]
    widths = [16, 16, 16, 16, 16, 16, 16, 16, 16, 16]

    # gen k=9 anchor size (h,w)
    heights = np.array(heights).reshape(len(heights), 1)
    widths = np.array(widths).reshape(len(widths), 1)

    base_anchors = np.array([0,0,15,15])


    # center of each cell
    xt = (base_anchors[0] + base_anchors[2]) / 2
    yt = (base_anchors[1] + base_anchors[3]) / 2

    # 10 base anchors, each has 4 coordinates x1, y1, x2, y2
    x1 = xt - widths / 2
    y1 = yt - heights / 2
    x2 = xt + widths / 2
    y2 = yt + heights / 2
    base_anchors = np.hstack((x1, y1, x2, y2))
    print('base_anchors: ', base_anchors)
    
    # shift the base anchors to the feature map
    h, w = feature_size
    shift_x = np.arange(0, w)*scale
    shift_y = np.arange(0, h)*scale
    print('shift_x: ', shift_x, 'shift_y: ', shift_y, 'scale: ', scale)
    # generate all anchors
    anchor = []
    for i in shift_y:
        for j in shift_x:
            anchor.append(base_anchors + [j, i, j, i])
    print('(base_anchors + [j, i, j, i]: ', (base_anchors + [j, i, j, i]))
    # print(np.array(anchor).reshape((-1, 4)))
    return np.array(anchor).reshape((-1, 4))

# 
def iou(anchor, gt):
    x1 = max(anchor[0], gt[0])
    y1 = max(anchor[1], gt[1])
    x2 = min(anchor[2], gt[2])
    y2 = min(anchor[3], gt[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    union = (anchor[2] - anchor[0]) * (anchor[3] - anchor[1]) + (gt[2] - gt[0]) * (gt[3] - gt[1]) - inter
    return inter / union


def cal_overlap(base_anchors, gtboxes):
    """
    计算anchors和ground truth boxes之间的IOU
    
    Args:
        base_anchors: [N, 4] anchor boxes (x1, y1, x2, y2)
        gtboxes: [M, 4] ground truth boxes (x1, y1, x2, y2)
    
    Returns:
        overlaps: [N, M] IOU矩阵
    """
    gtboxes = np.array(gtboxes)
    
    # 计算所有anchor和gt box之间的IOU
    overlaps = np.zeros((base_anchors.shape[0], gtboxes.shape[0]))
    print('overlaps.shape: ', overlaps.shape)
    
    for k in range(gtboxes.shape[0]):
        gt_box = gtboxes[k, :]  # 当前GT box [x1, y1, x2, y2]
        
        # 计算交集区域的坐标
        # 交集左上角：max(anchor_x1, gt_x1), max(anchor_y1, gt_y1)
        # 交集右下角：min(anchor_x2, gt_x2), min(anchor_y2, gt_y2)
        # print(base_anchors[:, 1], gt_box[1])
        inter_x1 = np.maximum(base_anchors[:, 0], gt_box[0])
        inter_y1 = np.maximum(base_anchors[:, 1], gt_box[1])
        inter_x2 = np.minimum(base_anchors[:, 2], gt_box[2])
        inter_y2 = np.minimum(base_anchors[:, 3], gt_box[3])
        
        # print('inter_x1: ', inter_x1, '\n', 'inter_x2: ', inter_x2, '\n', 'inter_y1: ', inter_y1, '\n', 'inter_y2: ', inter_y2)

        # 计算交集面积
        inter_w = np.maximum(0.0, inter_x2 - inter_x1)
        inter_h = np.maximum(0.0, inter_y2 - inter_y1)
        # print('inter_w: ', inter_w, '\n', 'inter_h: ', inter_h)
        intersection = inter_w * inter_h

        # 计算anchor和gt box的面积
        anchor_areas = (base_anchors[:, 2] - base_anchors[:, 0]) * \
                      (base_anchors[:, 3] - base_anchors[:, 1])
        # print('anchor_areas: ', anchor_areas)
        gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
        
        # 计算并集面积
        union = anchor_areas + gt_area - intersection
        # print('union: ', union, 'union.shape: ', union.shape)
        # 计算IOU，避免除零
        # print('intersection: ', intersection, 'intersection.shape: ', intersection.shape)
        overlaps[:, k] = intersection / (union + 1e-6)
        # print('overlaps: ', sorted(overlaps[:, k]))
    
    return overlaps


def bbox_transform(base_anchors, gtboxes):
    """
    计算从anchor box到ground truth box的回归目标
    
    Args:
        base_anchors: [N, 4] anchor boxes (x1, y1, x2, y2)
        gtboxes: [N, 4] ground truth boxes (x1, y1, x2, y2)
    
    Returns:
        bbox_targets: [N, 4] 回归目标 (dx, dy, dw, dh)
    """
    
    # 确保输入是numpy数组
    base_anchors = np.array(base_anchors, dtype=np.float32)
    gtboxes = np.array(gtboxes, dtype=np.float32)
    
    # 从边界坐标计算中心点坐标和宽高
    # Anchor boxes的中心点和宽高
    anchor_widths = base_anchors[:, 2] - base_anchors[:, 0] + 1.0
    anchor_heights = base_anchors[:, 3] - base_anchors[:, 1] + 1.0
    anchor_ctr_x = base_anchors[:, 0] + 0.5 * (anchor_widths - 1.0)
    anchor_ctr_y = base_anchors[:, 1] + 0.5 * (anchor_heights - 1.0)
    
    # Ground truth boxes的中心点和宽高
    gt_widths = gtboxes[:, 2] - gtboxes[:, 0] + 1.0
    gt_heights = gtboxes[:, 3] - gtboxes[:, 1] + 1.0
    gt_ctr_x = gtboxes[:, 0] + 0.5 * (gt_widths - 1.0)
    gt_ctr_y = gtboxes[:, 1] + 0.5 * (gt_heights - 1.0)
    
    # 计算回归目标
    # dx, dy: 中心点的相对偏移（归一化到anchor的宽高）
    dx = (gt_ctr_x - anchor_ctr_x) / anchor_widths
    dy = (gt_ctr_y - anchor_ctr_y) / anchor_heights
    
    # dw, dh: 宽高的对数缩放比例
    dw = np.log(gt_widths / anchor_widths)
    dh = np.log(gt_heights / anchor_heights)
    
    # 组合成回归目标 [dx, dy, dw, dh]
    bbox_targets = np.column_stack((dx, dy, dw, dh))
    
    return bbox_targets


def cal_rpn(img_size, feature_size, scale, gtboxes):
    h, w = img_size

    print('img_size: ', img_size, '\n', 'feature_size: ', feature_size, '\n', 'scale: ', scale)
    # gen base anchors
    base_anchors = gen_anchors(feature_size, scale)
    
    print('base_anchors.shape: ', base_anchors.shape)
    # print('base_anchors.shape: ', base_anchors.shape)
    # print('gtboxes: ', gtboxes)
    # calculate iou between gtboxes and base_anchors
    overlaps = cal_overlap(base_anchors, gtboxes)
    print('overlaps.shape: ', overlaps.shape)
    # init labels -1 don't care, 0 background, 1 foreground
    labels = np.empty(base_anchors.shape[0])
    labels.fill(-1)

    # for each gtbox,corresponds to an anchor which has highest iou
    # the ahchor with the highest iou overlap with a gtbox
    anchor_argmax_overlaps = overlaps.argmax(axis=0)
    # print('anchor_argmax_overlaps: ', anchor_argmax_overlaps)
    anchor_max_overlaps = overlaps[anchor_argmax_overlaps, range(overlaps.shape[1])]
    print('anchor_max_overlaps: ', anchor_max_overlaps)
    # iou > IOU_POSITIVE, label = 1
    labels[anchor_max_overlaps >= config.IOU_POSITIVE] = 1
    # iou < IOU_NEGATIVE, label = 0
    labels[anchor_max_overlaps < config.IOU_NEGATIVE] = 0
    # ensure that each gtbox has at least one foreground anchor
    labels[anchor_argmax_overlaps] = 1

    # check anchors outside image
    outside_anchor = np.where(
        (base_anchors[:, 0] < 0) |  
        (base_anchors[:, 1] < 0) |
        (base_anchors[:, 2] > w) |
        (base_anchors[:, 3] > h)
    )[0]

    labels[outside_anchor] = -1

    # subsample foreground labels, if greater than RPN_FORE_NUM(default 128)
    fg_index = np.where(labels == 1)[0]
    if len(fg_index) > config.RPN_FORE_NUM:
        labels[np.random.choice(fg_index, len(fg_index) - config.RPN_FORE_NUM, replace=False)] = -1
    
    # subsample background labels
    # if not config.OHEM:
    #     bg_index = np.where(labels == 0)[0]
    #     num_bg = 
    #     if len(bg_index) > config.RPN_BACK_NUM:
    #         labels[np.random.choice(bg_index, len(bg_index) - config.RPN_BACK_NUM, replace=False)] = -1
    # print(anchor_max_overlaps)
    # print(gtboxes)
    # calculate bbox targets
    bbox_targets = bbox_transform(base_anchors, gtboxes[anchor_max_overlaps, :])


    return [labels, bbox_targets], base_anchors


In [203]:



# def bbox_transform_inv(bbox_deltas, base_anchors):
#     """
#     bbox_transform的逆变换，用于将回归预测转换回边界框坐标
    
#     Args:
#         bbox_deltas: [N, 4] 回归预测 (dx, dy, dw, dh)
#         base_anchors: [N, 4] anchor boxes (x1, y1, x2, y2)
    
#     Returns:
#         pred_boxes: [N, 4] 预测的边界框 (x1, y1, x2, y2)
#     """
    
#     base_anchors = np.array(base_anchors, dtype=np.float32)
#     bbox_deltas = np.array(bbox_deltas, dtype=np.float32)
    
#     # 计算anchor的中心点和宽高
#     anchor_widths = base_anchors[:, 2] - base_anchors[:, 0] + 1.0
#     anchor_heights = base_anchors[:, 3] - base_anchors[:, 1] + 1.0
#     anchor_ctr_x = base_anchors[:, 0] + 0.5 * (anchor_widths - 1.0)
#     anchor_ctr_y = base_anchors[:, 1] + 0.5 * (anchor_heights - 1.0)
    
#     # 提取回归量
#     dx = bbox_deltas[:, 0]
#     dy = bbox_deltas[:, 1]
#     dw = bbox_deltas[:, 2]
#     dh = bbox_deltas[:, 3]
    
#     # 计算预测的中心点和宽高
#     pred_ctr_x = dx * anchor_widths + anchor_ctr_x
#     pred_ctr_y = dy * anchor_heights + anchor_ctr_y
#     pred_w = np.exp(dw) * anchor_widths
#     pred_h = np.exp(dh) * anchor_heights
    
#     # 转换回边界框坐标
#     pred_x1 = pred_ctr_x - 0.5 * (pred_w - 1.0)
#     pred_y1 = pred_ctr_y - 0.5 * (pred_h - 1.0)
#     pred_x2 = pred_ctr_x + 0.5 * (pred_w - 1.0)
#     pred_y2 = pred_ctr_y + 0.5 * (pred_h - 1.0)
    
#     pred_boxes = np.column_stack((pred_x1, pred_y1, pred_x2, pred_y2))
    
#     return pred_boxes

In [224]:

class ICDARDataset():
    def __init__(self, img_dir, gt_dir):
        self.img_dir = img_dir
        self.gt_dir = gt_dir
        self.img_list = os.listdir(img_dir)
        self.img_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x)) or 0))
        self.gt_list = os.listdir(gt_dir)
        self.gt_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x)) or 0))
    
    def __len__(self):
        """返回数据集大小"""
        return len(self.img_list)

        
    def __getitem__(self, index):
        """根据索引返回一个样本"""
        img_name = self.img_list[index]
        img_path = os.path.join(self.img_dir, img_name)

        img = cv2.imread(img_path)

        if img is None:
            print(img_path)
            with open('error_img.txt', 'a') as f:
                f.write('{}\n'.format(img_path))
            img_name = 'img_4929.jpg'
            img_path = os.path.join(self.img_dir, img_name)
            img = cv2.imread(img_path)

        h, w, c = img.shape
        # 缩放图片至1600
        rescale_fac = max(h, w) / 1600
        if rescale_fac > 1:
            h = int(h/rescale_fac)
            w = int(w/rescale_fac)
            img = cv2.resize(img, (w, h))

        # 读取标注文件
        
        try:
            gt_path = os.path.join(self.gt_dir, ('gt_'+img_name).replace('.png', '.txt'))
            gtbox = self.parse_gtfile(gt_path, rescale_fac)
        except FileNotFoundError:
            gt_path = os.path.join(self.gt_dir, ('gt_'+img_name).replace('.jpg', '.txt'))
            gtbox = self.parse_gtfile(gt_path, rescale_fac)
        # print(img_name)
        print('gtbox: ', gtbox)
        # random flip 数据增强
        # if np.random.randint(2) == 1:
        #     img = img[:, ::-1, :]
        #     newx1 = w - gtbox[:, 2] - 1
        #     newx2 = w - gtbox[:, 0] - 1
        #     gtbox[:, 0] = newx1
        #     gtbox[:, 2] = newx2
         
        # 生成RPN标签
        [cls, regr], base_anchors = cal_rpn(
            (h, w),
            (int(h/16), int(w/16)),
            16,
            gtbox
            )
        
        # 
        m_img = img - [123.68, 116.78, 103.94]
        # print(regr)
        regr = np.hstack([cls.reshape(cls.shape[0], 1), regr])
        cls = np.expand_dims(cls, axis=0)

        # transform to tensor
        # img = torch.from_numpy(img.transpose([2, 0, 1])).float()
        m_img = torch.from_numpy(m_img.transpose([2, 0, 1])).float()
        regr = torch.from_numpy(regr).float()
        cls = torch.from_numpy(cls).float()

        return img, cls, regr
    
    # 解析标注文件
    def parse_gtfile(self, gt_path, rescale_fac):
            print('rescale_fac: ', rescale_fac)
            with open(gt_path, 'r') as f:
                gt = f.readlines()
            gtbox = []
            for line in gt:
                # 切割成
                line = line.strip().split(',')
                # print(line)
                x1, y1, x2, y2 = float(line[0]), float(line[1]), float(line[4]), float(line[5])
                
                x1 = x1 / rescale_fac
                y1 = y1 / rescale_fac
                x2 = x2 / rescale_fac
                y2 = y2 / rescale_fac
                # print(x1, y1, x2, y2)
                gtbox.append([x1, y1, x2, y2])
            gtbox = np.array(gtbox)
            

            
            return gtbox


In [225]:
reload(config)

<module 'config' from 'f:\\projects\\OCR\\train\\train_ctpn\\config.py'>

In [226]:
dataset = ICDARDataset(config.icdar17_mlt_img_dir, config.icdar17_mlt_gt_dir)

In [227]:
# 直接索引访问（这会调用 __getitem__）
sample = dataset[7159]  # 获取第一个样本
# img, cls, regr = dataset[0]  # 解包返回值

rescale_fac:  1.28
rescale_fac:  1.28
gtbox:  [[ 487.5      601.5625   545.3125   649.21875]
 [ 688.28125  609.375    801.5625   647.65625]
 [ 435.15625  428.90625  534.375    456.25   ]
 [ 550.78125  430.46875  707.8125   455.46875]
 [ 729.6875   425.       850.       459.375  ]
 [ 832.03125  329.6875  1050.       428.125  ]
 [ 365.625    596.09375  423.4375   647.65625]
 [ 862.5      435.15625  957.03125  459.375  ]
 [ 337.5      331.25     800.78125  428.125  ]
 [ 551.5625   600.78125  683.59375  651.5625 ]
 [ 434.375    610.9375   488.28125  651.5625 ]
 [ 801.5625   607.03125  902.34375  649.21875]
 [ 894.53125  607.03125  953.125    651.5625 ]
 [ 961.71875  613.28125 1032.8125   650.     ]
 [1331.25     609.375   1379.6875   672.65625]
 [1382.03125  617.96875 1424.21875  667.96875]
 [1424.21875  613.28125 1457.03125  666.40625]
 [1460.15625  613.28125 1592.1875   667.96875]
 [1426.5625   562.5     1475.       596.875  ]
 [1485.15625  563.28125 1592.1875   600.78125]
 [1335.15625  

IndexError: boolean index did not match indexed array along axis 0; size of axis is 56000 but size of corresponding boolean axis is 21